In [1]:
import nibabel as nib
import numpy as np


Generate L/R hemisphere masks.

In [2]:

#-----------------------------------------------------------------------------
# Load the MNI152 template
#-----------------------------------------------------------------------------
template_path = "mni152.nii.gz"  # Update with your template path
template_img = nib.load(template_path)
template_data = template_img.get_fdata()  # We won't really use it, but we need its shape
affine = template_img.affine

#-----------------------------------------------------------------------------
# 2. Create a "non-zero" mask from the template
#-----------------------------------------------------------------------------
nonzero_mask = template_data != 0

#-----------------------------------------------------------------------------
# 3. Generate voxel indices and convert to real-world MNI coordinates
#-----------------------------------------------------------------------------
dims = template_data.shape
i, j, k = np.meshgrid(
    np.arange(dims[0]),
    np.arange(dims[1]),
    np.arange(dims[2]),
    indexing="ij"
)

voxel_coords = np.column_stack((i.flatten(), j.flatten(), k.flatten()))
mni_coords = nib.affines.apply_affine(affine, voxel_coords)
x_coords = mni_coords[:, 0]

#-----------------------------------------------------------------------------
# 4. Create left and right hemisphere masks
#-----------------------------------------------------------------------------
left_mask = np.zeros(dims, dtype=np.uint8)
right_mask = np.zeros(dims, dtype=np.uint8)

left_voxels = x_coords < 0
right_voxels = x_coords > 0

left_mask[
    voxel_coords[left_voxels, 0],
    voxel_coords[left_voxels, 1],
    voxel_coords[left_voxels, 2]
] = 1

right_mask[
    voxel_coords[right_voxels, 0],
    voxel_coords[right_voxels, 1],
    voxel_coords[right_voxels, 2]
] = 1

#-----------------------------------------------------------------------------
# 5. Restrict masks to only the non-zero region in the template
#-----------------------------------------------------------------------------
left_mask = left_mask * nonzero_mask
right_mask = right_mask * nonzero_mask

#-----------------------------------------------------------------------------
# 6. Save out the NIfTI masks
#-----------------------------------------------------------------------------
left_mask_img = nib.Nifti1Image(left_mask, affine)
right_mask_img = nib.Nifti1Image(right_mask, affine)

nib.save(left_mask_img, "left_hem_mask.nii.gz")
nib.save(right_mask_img, "right_hem_mask.nii.gz")

print("Left/right hemisphere masks saved. They now only include non-zero template voxels.")

Left/right hemisphere masks saved. They now only include non-zero template voxels.


Mask bigram and lexical frequency correlation contrasts with L/R masks. Specifically, divide right-hemisphere by left hemispheric activation.

In [3]:
from pathlib import Path
lexical_tmap = Path('/home/neel/Documents/SPM_results/SPM-V_II_Zipf_multireg (FINAL)/spmT_0001.nii')
lex_tmap = nib.load(lexical_tmap)
lex_data = lex_tmap.get_fdata()
lex_affine = lex_tmap.affine

# #masking the lex_data with left and right hemisphere masks
# left_lex_data = lex_data * left_mask
# right_lex_data = lex_data * right_mask

# #divide right by left
# right_by_left = right_lex_data / left_lex_data
# #save this out as a nifti file
# right_by_left_img = nib.Nifti1Image(right_by_left, lex_affine)
# nib.save(right_by_left_img, "right_by_left.nii.gz")
# print("Right by left hemisphere mask saved.")


In [4]:
#divide every value in the lex_data by the maximum T-value of lex_data
max_lex_data = np.max(lex_data)
lex_data = lex_data / max_lex_data
#save this out as a nifti file
lex_data_img = nib.Nifti1Image(lex_data, lex_affine)
nib.save(lex_data_img, "visual_word_tmap_pct_vwfa.nii.gz")


In [5]:

bigram_tmap = Path('/home/neel/Documents/SPM_results/SPM-V_II_Lg10BG_multireg_november (FINAL)/spmT_0001.nii')
bigram_tmap = nib.load(bigram_tmap)
bigram_data = bigram_tmap.get_fdata()
bigram_affine = bigram_tmap.affine

#do the same thing as above: divide every value in the bigram_data by the maximum T-value of bigram_data
max_bigram_data = np.max(bigram_data)
bigram_data = bigram_data / max_bigram_data
#save this out as a nifti file
bigram_data_img = nib.Nifti1Image(bigram_data, bigram_affine)
nib.save(bigram_data_img, "visual_bigram_tmap_pct_op.nii.gz")